# 질문과 답변의 키워드 추출

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import krwordrank

/var/folders/z6/xj7t2mkx4pdb7lbxyjn0tbb00000gn/T/ipykernel_36630/3227209871.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [13]:
DATA_PATH = '/Users/jungnerd/ML/001.Projects/DACON_hansol_deco/dataset/' 
train_df = pd.read_csv(DATA_PATH + 'train.csv') 
test_df = pd.read_csv(DATA_PATH + 'test.csv') 
submission_df = pd.read_csv(DATA_PATH + 'sample_submission.csv')

In [14]:
train_df.head()

,id,질문_1,질문_2,category,답변_1,답변_2,답변_3,답변_4,답변_5
0,TRAIN_000,면진장치가 뭐야?,면진장치에 사용되는 주요 기술은 무엇인가요?,건축구조,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을...",면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...
1,TRAIN_001,내진설계의 종류 좀 알려줘,내진설계에는 어떤 종류가 있는지 자세히 알려주실 수 있나요?,건축구조,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계에는 내진구조, 제진구조, 면진구조가 있습니다. 내진구조는 건물 구조물이 지...","내진설계에는 주로 내진구조, 제진구조, 면진구조의 세 가지 종류가 있습니다. 이들은...","내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조...","내진 설계에는 다양한 종류가 있지만, 대표적으로 내진구조, 제진구조, 면진구조가 있..."
2,TRAIN_002,철골구조의 장점이 뭐야?,철골구조의 장점을 알려줘?,건축구조,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점은 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건...,철골구조의 장점은 건물의 외벽에 하중이 적게 걸리기 때문에 고층 건물의 건축이 용이...,"철골구조의 장점은 건물의 외벽이 하중이 걸리지 않아 공간 활용이 용이하고, 고층 건...",철골구조의 장점은 건물의 외벽에 하중이 크게 걸리지 않아 고층 건물을 건축할 수 있...
3,TRAIN_003,철골철근 콘크리트 구조가 뭐야?,철골철근 콘크리트 구조의 장점과 단점에는 무엇이 있을까요?,건축구조,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ...","철골철근콘크리트 구조는 건축물을 지탱하는 주요 구조물인 철골과 철근, 그리고 콘크리...",철골철근 콘크리트 구조는 건축물을 지탱하기 위한 구조물에서 일반적으로 사용되는 방식...,"철골철근콘크리트 구조는 철골과 철근, 그리고 콘크리트를 함께 사용하여 만들어지는 건...","철골철근 콘크리트 구조는 강철 골조와 강철 철근, 그리고 콘크리트를 함께 사용하여 ..."
4,TRAIN_004,철골구조는 어떤 방식이 있어?,철골구조의 다양한 방식이 무엇인가요?,건축구조,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 일반철골구조와 경량철골구조가 있습니다. 일반철골구조는 주로 대형 건물이나...,철골구조는 주로 일반철골구조와 경량철골구조로 나뉘어집니다. 이들은 건축 시스템에 따...,철골구조는 주로 일반철골구조와 경량철골구조로 구분됩니다. 이외에도 최근에는 고층 건...,철골구조는 일반철골구조와 경량철골구조 두 가지 방식이 주로 사용됩니다. 일반철골구조...


In [19]:
question_1_list = train_df['질문_1'].tolist()
question_2_list = train_df['질문_2'].tolist()
print(question_1_list[0:5])
print(question_2_list[0:5])

['면진장치가 뭐야?', '내진설계의 종류 좀 알려줘', '철골구조의 장점이 뭐야?', '철골철근 콘크리트 구조가 뭐야?', '철골구조는 어떤 방식이 있어?']
['면진장치에 사용되는 주요 기술은 무엇인가요?', '내진설계에는 어떤 종류가 있는지 자세히 알려주실 수 있나요?', '철골구조의 장점을 알려줘?', '철골철근 콘크리트 구조의 장점과 단점에는 무엇이 있을까요?', '철골구조의 다양한 방식이 무엇인가요?']


## krwordrank 기본 활용

In [20]:
from krwordrank.word import summarize_with_keywords

keywords_1 = summarize_with_keywords(question_1_list)
print(keywords_1)

{'뭐야?': 29.325890397183965, '어떻게': 8.207116434994589, '장점이': 5.132308226873308, '어떤': 4.790283034020434, '단점이': 4.670136807670311, '있어?': 4.48068454711637, '인테리어': 4.09856593946723, '무엇인가요?': 3.740183586336049, '얼마야?': 3.410686673126313, '대해': 3.4074805748342594, '도배': 2.7777964146150507, '하나요?': 2.6499794043613303, '인해': 2.2066124908009495, '뭐가': 1.8269827424838008, '알려줘': 1.6751496640940426, '발생할': 1.6480768159403825, '타일의': 1.6103515663643164, '있을': 1.4459490775187702, '결로의': 1.3940078661172999, '사용되는': 1.3490482547427423, '꾸밀': 1.3228879500594108, '공간': 1.3048042010667347, '벽면에': 1.2735969578096924, '디자인': 1.271080688182145, '기대수명은': 1.2648242687388596, '위한': 1.2515542351754565, '설명해줘': 1.224882943019118, '페인': 1.2119878736127925, '경우': 1.1408470012322869, '어디에': 1.1312627480999236, '해?': 1.123140773391591, '조명을': 1.0878488970228695, '벽지': 1.0840345688273958, '종류': 1.0603816869792524, '보온판': 1.0132080763248226, '활용하여': 1.0087134890098386, '실내': 0.9925092516801176, '비드법': 0.97858344

In [21]:
keywords_2 = summarize_with_keywords(question_2_list)
keywords_2

{'어떤': 19.29080521314884,
 '무엇인가요?': 13.494977956491667,
 '있을까요?': 7.900803526755514,
 '있나요?': 5.374574965481748,
 '알려주세요.': 4.268113639976229,
 '사용': 4.01342728778337,
 '것이': 3.395470544171265,
 '어떻게': 3.385148695124413,
 '장점은': 3.345136302352014,
 '도배': 3.2436346744391003,
 '가장': 3.0647941863938932,
 '대해': 3.0611599145844295,
 '알고': 2.975617584971986,
 '하나요?': 2.8971345460991893,
 '인테리어': 2.7974004189644903,
 '있는': 2.683562252014568,
 '할까요?': 2.5350521696581154,
 '방법': 2.314742036948056,
 '하는': 2.277565988331284,
 '단점은': 2.0272059170145202,
 '인해': 1.8965111322269594,
 '종류': 1.855463615737404,
 '것들이': 1.8477767027873742,
 '자세히': 1.8112175642566317,
 '위한': 1.8082354972012022,
 '위해': 1.7764210946471009,
 '좋을까요?': 1.7245082051495078,
 '용도로': 1.66625783426068,
 '건가요?': 1.572692863101345,
 '효과적인': 1.3724139287436126,
 '경우': 1.352478978248136,
 '얼마나': 1.3305227659500427,
 '디자인': 1.2776905898366704,
 '것의': 1.223093296270949,
 '재료로': 1.2016212911683546,
 '발생': 1.1142019621593815,
 '되나요?': 1.0

키워드가 아니지만 자주 반복되는 단어를 키워드로 뽑는 경향이 보인다.

## 문장 전처리

### 명사로만 문장을 구성하도록

In [17]:
from konlpy.tag import Okt
import re

def split_noun_sentence(sentences):
    okt = Okt()
    # sentences = text.replace(". ",".")
    # sentences = re.sub(r'([^\n\s\.\?!]+[^\n\.\?!]*[\.\?!])', r'\1\n', sentences).strip().split("\n")
    
    result = []
    for sentence in sentences:
        if len(sentence) == 0:
            continue
        sentence_pos = okt.pos(sentence, stem=True)
        nouns = [word for word, pos in sentence_pos if pos == 'Noun']
        if len(nouns) == 1:
            continue
        result.append(' '.join(nouns) + '.')
        
    return result

In [18]:
split_noun_sentence(question_1_list)

['면 진장 치가 뭐.',
 '진설 종류 좀.',
 '철 조의 장점 뭐.',
 '철골 철근 콘크리트 구조 뭐.',
 '철골 구조 방식.',
 '커튼월 뭐.',
 '진구 뭐.',
 '중목 구조 방식 뭐.',
 '기둥 보 구조 방식 뭐.',
 '블록 구조 뭐.',
 '철 조의 단점 뭐.',
 '콘크리트 구조 방식.',
 '철골 콘크리트 구조 뭐.',
 '면 진구 뭐.',
 '철 조가 뭐.',
 '면 진구 진구 차이점 뭐.',
 '철근콘크리트 구조 뭐.',
 '건축물 구조 구분.',
 '조적 구조 방식.',
 '목 구조 방식.',
 '경량 철 조가 뭐.',
 '구조 뭐.',
 '화재 취약 철 조의 보완 법 뭐.',
 '프리 케스트 콘크리트 구조 뭐.',
 '진구 뭐.',
 '통나무 구조 방식 뭐.',
 '철 조가 뭐.',
 '진설 안정 순서대로 말.',
 '경량 목 구조 방식 뭐.',
 '조적 구조 뭐.',
 '벽돌 구조 뭐.',
 '아세톤 노출 어디가.',
 '주택 실내 라돈 측정 지점.',
 '소화기 설치 기준 뭐.',
 '아파트 도배 평수 게.',
 '방수 발 수의 차이 뭐.',
 '건설 뭐.',
 '토목 뭐.',
 '층간소음 기준 뭐.',
 '음 뭐.',
 '새집 증후군 뭐.',
 '라돈 제외 주택 실내 기질 측정 지점.',
 '사무실 실내 공기 시료 채취 및 측정 지점.',
 '소화기 종류 뭐.',
 '새집 증후군 예방 방법 뭐.',
 '분말 소화기 뭐.',
 '건축 뭐.',
 '마감 재의 하자 여부 판단.',
 '분말 소화기 사용 때 주의 점 뭐.',
 '새집 증후군 원인 뭐.',
 '소화기 뭐.',
 '스마트홈 시스템 집 설치 보안 대해 조치 취해 해.',
 '이산화탄소 소화기 뭐.',
 '방음 뭐.',
 '새집 증후군 해결 방법 뭐.',
 '주택 도배 평수 계산.',
 '원목 마루 단점 뭐.',
 '롱 브릭 타일 뭐.',
 '카페트 기대수명 얼마.',
 '평 지붕 단점 뭐.',
 '개별 공간 오픈 플랜 공간 중 것 나 더.',
 '도배지 처리.',
 

In [22]:
split_noun_sentence(question_2_list)

['면진 장치 사용 주요 기술 무엇.',
 '진설 종류 수.',
 '철 조의 장점.',
 '철골 철근 콘크리트 구조 장점 단점 무엇.',
 '철 조의 방식 무엇.',
 '커튼월 무엇.',
 '진구 대해.',
 '중목 구조 방식 대해 좀 더 알.',
 '기둥 보 구조 방식 무엇.',
 '블록 구조 무엇.',
 '철 조의 장점.',
 '콘크리트 구조 데 사용 주요 방식 것.',
 '철골 콘크리트 구조 란 무엇.',
 '면 진구 대해 좀 더 알 수.',
 '철골 구조 대해 좀 더.',
 '면 진구 진구 차이점 대해.',
 '철근콘크리트 구조 특징 장점 무엇.',
 '건축물 구조 구분 데 기준 활용 하나요.',
 '조적 구조 방식 대해.',
 '목 구조 종류 방식.',
 '경량 철골 구조 구조.',
 '구조 대해.',
 '철 조의 화재 대비 보완 법 것.',
 '프리 케스트 콘크리트 구조 란 무엇.',
 '진구 무엇.',
 '통나무 구조 방식 무엇.',
 '철 무엇.',
 '진설 안전성 위 순서 무엇.',
 '경량 목 구조 방식 무엇.',
 '조적 구조 대해 좀 더 설명 수.',
 '벽돌 구조.',
 '아세톤 노출 발생 수 문제 뭐.',
 '주택 실내 라돈 측정 지점 어디 정 것 가장.',
 '소화기 설치 기준 무엇.',
 '아파트 도배 평수 계산 하나요.',
 '방수 발 수의 차이점 무엇.',
 '건설업 종류 활동 포함 하나요.',
 '토목 무엇.',
 '층간소음 발생 기준 것.',
 '음 무엇.',
 '새집 증후군 대해 좀 더.',
 '주택 실내 기질 측정 위 지점 설정 게.',
 '사무실 실내 공기 측정 위해 장비.',
 '종류 소화기.',
 '새집 증후군 예방 위해 사용 수 실용 방법.',
 '분말 소화기 용도 사용.',
 '건축 것 의미 하나요.',
 '마감 재의 하자 확인 수.',
 '분말 소화기 사용 때 주의 사항.',
 '새집 증후군 원리 발생 하나요.',
 '소화기 란 무엇.',
 '스마트홈 시스템 집 설치 때 보안 조치 취해.',
 '무엇 설명.',

인테리어 관련 용어를 명사로 인식하지 못함...  
좋은 방법이 없을까?